<a href="https://colab.research.google.com/github/NikoDhrmwn/PadE/blob/main/PadE_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [2]:
train_dir="/content/drive/MyDrive/rice-leaf-desease/training"
val_dir="/content/drive/MyDrive/rice-leaf-desease/validation"
Labels=[]

for file in os.listdir(train_dir):
    Labels+=[file]

Label=[]

for i in range(len(Labels)):
    Label+=[i]
    
mapping = dict(zip(Labels, Label)) 
reverse_mapping = dict(zip(Label, Labels)) 

def mapper(value):
    return reverse_mapping[value]

dataset=[]
valset=[]
count=0
for file in os.listdir(train_dir):
    path=os.path.join(train_dir,file)
    t=0
    for im in os.listdir(path):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(32,32))
        image=img_to_array(image)
        image=image/255.0
        dataset+=[[image,count]]
        t+=1
    count=count+1
count=0
for file in os.listdir(val_dir):
    path=os.path.join(val_dir,file)
    t=0
    for im in os.listdir(path):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(32,32))
        image=img_to_array(image)
        image=image/255.0
        valset+=[[image,count]]
        t+=1
    count=count+1

In [3]:
data,labels0=zip(*dataset)
test,testlabels0=zip(*valset)
labels1=to_categorical(labels0)
labels=np.array(labels1)
data=np.array(data)
test=np.array(test)
trainx,testx,trainy,testy=train_test_split(data,labels,test_size=0.2,random_state=44)

print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)

(12640, 32, 32, 3)
(3160, 32, 32, 3)
(12640, 4)
(3160, 4)


In [4]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.1,
                             width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,fill_mode="nearest")

In [5]:
model = tf.keras.models.Sequential([ 
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256,activation = 'relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(512,activation = 'relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1024,activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
  ])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='categorical_crossentropy',
                metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1152)              0         
                                                                 
 dense (Dense)               (None, 128)               1

In [6]:
history = model.fit(datagen.flow(trainx,trainy,batch_size=75),validation_data=(testx,testy),epochs=35)

Epoch 1/35
169/169 [==============================] - 13s 73ms/step - loss: 0.8882 - accuracy: 0.5515 - val_loss: 0.7118 - val_accuracy: 0.6994
Epoch 2/35
169/169 [==============================] - 12s 71ms/step - loss: 0.6385 - accuracy: 0.7205 - val_loss: 0.4327 - val_accuracy: 0.8177
Epoch 3/35
169/169 [==============================] - 12s 72ms/step - loss: 0.4383 - accuracy: 0.8217 - val_loss: 0.2998 - val_accuracy: 0.8756
Epoch 4/35
169/169 [==============================] - 12s 71ms/step - loss: 0.3216 - accuracy: 0.8671 - val_loss: 0.1832 - val_accuracy: 0.9237
Epoch 5/35
169/169 [==============================] - 12s 72ms/step - loss: 0.3031 - accuracy: 0.8790 - val_loss: 0.2245 - val_accuracy: 0.9196
Epoch 6/35
169/169 [==============================] - 12s 72ms/step - loss: 0.2326 - accuracy: 0.9062 - val_loss: 0.1633 - val_accuracy: 0.9329
Epoch 7/35
169/169 [==============================] - 12s 71ms/step - loss: 0.2217 - accuracy: 0.9122 - val_loss: 0.1738 - val_accuracy:

In [9]:
image=load_img("/content/BacterialLeafBlight.jpg", target_size=(32,32))

image=img_to_array(image) 
image=image/255.0
prediction_image=np.array(image)
prediction_image= np.expand_dims(image, axis=0)
prediction=model.predict(prediction_image)
value=np.argmax(prediction)
move_name=mapper(value)
print("Prediction is {}.".format(move_name))

Prediction is Bacterial leaf blight.
